In [1]:
file=open("2019_08_Marvin1_without_labels_or_heading_orwidgets.txt")
dataset=file.read()
print (dataset[0:700])

 HAROLD MARCH, the rising reviewer and social critic, was walking vigorously across a great tableland of moors and commons, the horizon of which was fringed with the far-off woods of the famous estate of Torwood Park. 
 A special program at the planetarium on the campus of nearby Butler University was the Barnetts’ turning point. A rapt Jake stunned the audience with an astute question about Mars’ moons, and on the drive home, couldn’t stop talking about the solar system. Jake “hadn’t been missing after all,” she realized, just because he was unreachable. Unhindered, he had been busy working: His repetitive behavior, much as some autism researchers have speculated, reflected detail-oriented 


In [3]:
documents=dataset.split('\n') #have been trying \t  with less success
print (documents[10])
print(len(documents))

Ultimately, feelings can annoy us or delight us, but that is not what they are for. Feelings are for life regulation, providers 
1041


In [4]:
#i can use this to test a particular strip on 1 paragraph before applying it to all
documents[1000].strip('\n')
print (documents[1000])


The ideas behind Peircean pragmatism are how to think about signs and representations (semiosis); logically reason and handle new knowledge (abduction) and probabilities (induction); make economic research choices (pragmatic maxim


In [9]:
#my first ever bit of for and iffing in python, built incrementally using print!
#dont think I need this if I am looking at my kindle
for doc in documents:
  if len(doc)<20000:
    documents.remove(doc)

In [11]:
print(len(documents))
print (documents[100])

130
I have applied to the writing of these Memoirs a really paternal predilection; I would wish to be able to rise at the ghostly hour to correct the proofs: the dead go fast 


In [12]:
#from Tutorial 1: Corpora and Vector Spaces¶
from gensim import corpora, models, similarities

In [ ]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

from pprint import pprint  # pretty-printer
pprint(texts)

In [ ]:
#create dictionary, then map from ids to dictionary
dictionary = corpora.Dictionary(texts)
print(dictionary)
print(dictionary.token2id)

In [ ]:
#convert tokenized documents to vector
new_doc = "KBpedia is a comprehensive knowledge structure for promoting data interoperability and knowledge-based artificial intelligence, or KBAI. The KBpedia knowledge structure combines seven ‘core’ public knowledge bases "
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # only those words that match up are given a dimension

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]
for c in corpus:
    print(c)

From Quick start tutorial. 
Now that we have vectorized our corpus we can begin to transform it using models. We use model as an abstract term referring to a transformation from one document representation to another. In gensim documents are represented as vectors so a model can be thought of as a transformation between two vector spaces. The details of this transformation are learned from the training corpus.
One simple example of a model is tf-idf. The tf-idf model transforms vectors from the bag-of-words representation to a vector space where the frequency counts are weighted according to the relative rarity of each word in the corpus.
Let's initialize the tf-idf model, training it on our corpus and transforming the string "system minors":

In [ ]:
from gensim import models
# train the model
tfidf = models.TfidfModel(corpus)
# transform the "The first thing to do,' said Psmith, 'is to ascertain that such a place as Clapham Common really exists." string
tfidf[dictionary.doc2bow("I cannot believe something that does not understand system feedback".lower().split())]


In [ ]:
#now movwed onto Topic and Transformations tutorial
#apply tfidf to the trained corpus
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

In [ ]:
#OPTIONAL
#this model can now be applied to another corpus other than the training one, not just individaul documents
#i have not pulled in a second corpus but this is how you would do it
corpus2nd_tfidf = tfidf[corpus2nd]
for doc in corpus2nd_tfidf:
    print(doc)

In [20]:
#now applying an LSI to the first corpus, by working on top of its representation as a TFIDF
# here we have created a two dim LSI space, like Deerwesters 1990 example
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=50) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi


In [ ]:
#inspect the topics
lsi.print_topics(50)


In [ ]:
#higher number per topic indicates more close to that topic
for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print(doc)

moved onto Similarity search tutorial.
Now suppose a user typed in the query “Human computer interaction”. We would like to sort our corpus documents in decreasing order of relevance to this query. Unlike modern search engines, here we only concentrate on a single aspect of possible similarities—on apparent semantic relatedness of their texts (words). No hyperlinks, no random-walk static ranks, just a semantic extension over the boolean keyword match:

In [ ]:
doc = "I cannot believe something that does not understand system feedback"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

In addition, we will be considering cosine similarity to determine the similarity of two vectors. Cosine similarity is a standard measure in Vector Space Modeling, but wherever the vectors represent probability distributions, different similarity measures may be more appropriate.


To prepare for similarity queries, we need to enter all documents which we want to compare against subsequent queries. In our case, they are the same nine documents used for training LSI, converted to 2-D LSA space. But that’s only incidental, we might also be indexing a different corpus altogether.

In [24]:
index = similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it

In [ ]:
sims = index[vec_lsi] # perform a similarity query against the corpus
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

Cosine measure returns similarities in the range <-1, 1> (the greater, the more similar), so that the first document has a score of 0.99809301 etc.

With some standard Python magic we sort these similarities into descending order, and obtain the final answer to the query “Human computer interaction”:

In [ ]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims) # print sorted (document number, similarity score) 2-tuples

In [30]:
print (documents[10]) #putting in the number from the top of the list above

Are you starying here on the off-chance of another row? There will be none till the Southern Soudan is reoccupied by our troops. Mark that.


In [31]:
#now trying the same thing with tfidf only. Not sure if I have created index properly
index = similarities.MatrixSimilarity(corpus_tfidf)
sims = index[vec_lsi] 
print(list(enumerate(sims)))
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print ("\n")
print (sims)


[(0, -0.28670505), (1, -0.11364706), (2, 0.18208382), (3, -0.026342653), (4, 0.0), (5, -0.113511264), (6, 0.0016789734), (7, -0.020899296), (8, 0.047693133), (9, 0.00026297205), (10, 0.07622522), (11, 0.0), (12, -0.05266291), (13, -0.018896377), (14, 0.0047806744), (15, -0.11393608), (16, 0.037752964), (17, 0.008575946), (18, -0.105849504), (19, -0.058788367), (20, 0.0), (21, -0.055237595), (22, 0.0), (23, -0.0025837943), (24, -0.022241551), (25, 0.0011724434), (26, 0.008334677), (27, 0.07443505), (28, -0.026223354), (29, 0.0), (30, -0.035312925), (31, 0.069166005), (32, -0.028262423), (33, -0.047730926), (34, 0.02637633), (35, 0.055644646), (36, 0.020075727), (37, -0.081954084), (38, 0.0040368787), (39, -0.11119308), (40, 0.033288468), (41, 0.026674578), (42, -0.06816816), (43, -0.054938383), (44, -0.10582238), (45, -0.020957656), (46, -0.21955796), (47, 0.0), (48, -0.039441206), (49, 0.0), (50, 0.0), (51, 0.018376706), (52, 0.005001612), (53, 0.050661176), (54, 0.022845889), (55, -0.

In [33]:
print (documents[23])

‘If I’ve seen anything, God knows I couldn’t have seen it but for you, and I know that I couldn’t have said it except to you. You seemed to make everything clear for a minute; but I don’t practice what I preach. You would help me.... There are only us two in the world for all purposes, and—and you like to have me with you?’
